# 「分散RPCフレームワークを用いたパラメーターサーバーの実装」

【原題】Implementing a Parameter Server Using Distributed RPC Framework

【原著】[Rohan Varma](https://github.com/rohan-varma)

【元URL】https://pytorch.org/tutorials/intermediate/rpc_param_server_tutorial.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年11月28日

【チュトーリアル概要】

前提知識:
- [PyTorch Distributedについて](https://pytorch.org/tutorials/beginner/dist_overview.html)（日本語版6_1）
- [RPC APIドキュメント](https://pytorch.org/docs/master/rpc.html)


本チュートリアルでは、PyTorchの[分散RPCフレームワーク](https://pytorch.org/docs/stable/rpc.html)を用いた、パラメーターサーバーの簡単な実装例を解説します。

パラメーターサーバーフレームワークは、複数のサーバーが大規模な埋め込みテーブルなどのパラメーターを格納し、複数のトレーナーがパラメーターサーバに問い合わせることで、ほぼ最新のパラメータを取得することが出来るようにする枠組みです。

これらのトレーナーは、ローカルで訓練ループを実行し、ときおりパラメーターサーバーと同期して、最新のパラメータを取得します。

パラメーターサーバーを用いたアプローチ手法の詳細に関しては、[こちらの論文](https://www.cs.cmu.edu/~muli/file/parameter_server_osdi14.pdf) を確認してください。

---



本チュートリアルでは、分散RPCフレームワークと共に、複数のトレーナーでRPCを使って同一のパラメーターサーバーに対して通信を行い、さらに[RRef](https://pytorch.org/docs/stable/rpc.html#torch.distributed.rpc.RRef)を使用してリモートのパラメーターサーバーのインスタンス状態にアクセスする例を構築します。



各トレーナーは、分散自動微分を使用して複数のノードにまたがる自動微分グラフを合わせることにより、分散した状態で専用のバックワードパスを使用するようにします。

**注意:**

本チュートリアルでは、モデルを複数のマシンに分割したり、ネットワークトレーナーが異なるマシン上で管理されているパラメーターを受け取れるような、パラメーターサーバーを用いた訓練手法を実装する際に役立つ、分散RPCフレームワークの使い方を網羅します。

仮に、このようなケースではなく、モデルを多数のGPU上に複製する方法を探している場合は、[分散データ並列訓練入門](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html) （日本語版6_3）を参照してください。

また、強化学習やRNNのユースケースを網羅した[RPCのチュートリアル](https://pytorch.org/tutorials/intermediate/rpc_tutorial.html) （日本語版6_5）も用意されています。

まずは馴染みのある部分から始めましょう。

必要なモジュールをインポートし、MNISTのデータセット上で訓練を行うシンプルな畳み込みニューラルネットワークを定義します。

なお、下記のネットワークの大部分は、[pytorch/examples のリポジトリ](https://github.com/pytorch/examples/tree/master/mnist)で定義されているものから借用しています。

In [ ]:
import argparse
import os
import time
from threading import Lock

import torch
import torch.distributed.autograd as dist_autograd
import torch.distributed.rpc as rpc
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.distributed.optim import DistributedOptimizer
from torchvision import datasets, transforms

# --------- pytorch/examplesより、訓練対象のMNISTネットワーク -----

class Net(nn.Module):
    def __init__(self, num_gpus=0):
        super(Net, self).__init__()
        print(f"Using {num_gpus} GPUs to train")
        self.num_gpus = num_gpus
        device = torch.device(
            "cuda:0" if torch.cuda.is_available() and self.num_gpus > 0 else "cpu")
        print(f"Putting first 2 convs on {str(device)}")
        # 畳み込み層を1つ目のcudaデバイスに配置します。cudaデバイスが存在しない場合、CPUに配置します。
        self.conv1 = nn.Conv2d(1, 32, 3, 1).to(device)
        self.conv2 = nn.Conv2d(32, 64, 3, 1).to(device)
        # 2つ目のcudaデバイスが存在する場合、残りのネットワークを2つ目のcudaデバイスに配置します。
        if "cuda" in str(device) and num_gpus > 1:
            device = torch.device("cuda:1")

        print(f"Putting rest of layers on {str(device)}")
        self.dropout1 = nn.Dropout2d(0.25).to(device)
        self.dropout2 = nn.Dropout2d(0.5).to(device)
        self.fc1 = nn.Linear(9216, 128).to(device)
        self.fc2 = nn.Linear(128, 10).to(device)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)

        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        # 必要に応じて、テンソルを次のデバイスに移動します。
        next_device = next(self.fc1.parameters()).device
        x = x.to(next_device)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

次に、残りのスクリプトで役に立つ補助関数を定義しましょう。

次のコードでは、[rpc_sync](https://pytorch.org/docs/stable/rpc.html#torch.distributed.rpc.rpc_sync) と [RRef](https://pytorch.org/docs/stable/rpc.html#torch.distributed.rpc.RRef) を使用し、リモートノードに存在するオブジェクト上で任意のメソッドを呼び出す関数を定義しています。



また、2番目に定義している関数では、リモートオブジェクトへのハンドルを `rref` 引数で与え、その `rref` の所有ノード上でメソッドを実行します。（`rref.owner()`）

なお、呼び出し元のノードでは、`rpc_sync` の使用を通してこれらの処理を同期的に実行するため、レスポンスを受け取るまでは処理をブロックする必要があります。

In [ ]:
# --------- Helper Methods --------------------

# ローカルノード上では、最初の引数をRRefによって保有されている値としてメソッドを呼び出します。
# そして他の引数は、関数を呼び出すために使用される引数として渡されます。
# この関数はインスタンスメソッドを呼び出す際に役立ちます。
# methodには、クラスメソッドを含む任意の適当な関数を渡すことが可能です。
def call_method(method, rref, *args, **kwargs):
    return method(rref.local_value(), *args, **kwargs)

# RRefを引数に受け取り、このRRefが保有する値を基に実行したメソッドの結果を返します。
# この呼出は、RRefを保有するリモートノード上で行われ、与えられた引数をメソッドに渡します。
# 例：RRefが保有している値の型が Foo である場合、remote_method(Foo.bar, rref, arg1, arg2) は、
#    <foo_instance>.bar(arg1, arg2)をリモートノード上で呼び出し、結果を取得していることと等価です。
def remote_method(method, rref, *args, **kwargs):
    args = [method, rref] + list(args)
    return rpc.rpc_sync(rref.owner(), call_method, args=args, kwargs=kwargs)

これで、パラメーターサーバーを定義する準備ができました。

nn.Moduleをサブクラス化し、上記で定義したネットワークへのハンドルを保存します。

また、モデルを呼び出す前に、入力を移動する先として、使用するデバイスを入力デバイスとして保存します。

In [ ]:
# --------- Parameter Server --------------------
class ParameterServer(nn.Module):
    def __init__(self, num_gpus=0):
        super().__init__()
        model = Net(num_gpus=num_gpus)
        self.model = model
        self.input_device = torch.device(
            "cuda:0" if torch.cuda.is_available() and num_gpus > 0 else "cpu")

次に、フォワードパスを定義します。

なお、現在のところDistributed RPC Frameworkは、RPC経由の場合にCPUのテンソルのみの送信をサポートしているため、モデルの出力のデバイスに関わらず、出力をCPUに移動する点に注意してください。

呼び出し元と呼び出し先でデバイスが異なる（CPU/GPU）可能性を考慮して、今回は意図的にRPC経由でCUDAのテンソルを送信しないようにしますが、後のリリースでサポートされていることを願います。

In [ ]:
class ParameterServer(nn.Module):
    # ...
    def forward(self, inp):
        inp = inp.to(self.input_device)
        out = self.model(inp)
        # この出力は、RPC経由でフォワードされますが、1.5.0の時点ではCPUのテンソルしか受け付けていません。
        # そのため、テンソルをGPUのメモリに出し入れしなければなりません。
        out = out.to("cpu")
        return out

次に、訓練や検証に役立つ諸々の関数を定義していきます。

1つ目は、`get_dist_gradients` 関数です。

これは、分散自動微分のコンテクストID を基に `dist_autograd.get_gradients` API を呼び出し、分散自動微分エンジンによって計算された勾配を取得します。

分散自動微分に関する詳細な情報は、[分散自動微分のドキュメント](https://pytorch.org/docs/stable/rpc.html#distributed-autograd-framework) で確認できます。

なお、現在のところ、フレームワークはRPCを介したテンソルの送信のみをサポートしているため、 `dist_autograd.get_gradients` の実行結果として得られるdictオブジェクトを反復して、各テンソルからCPUテンソルへの変換も行っている点に注意してください。



次に、`get_param_rrefs` 関数は、モデルのパラメーターを反復し、（ローカルの）[RRef](https://pytorch.org/docs/stable/rpc.html#torch.distributed.rpc.RRef) としてそれらのパラメーターをラップします。

このメソッドは、RPC経由でトレーナーノードによって呼び出され、最適化の対象となるパラメーターのリストを返します。

このリストは、[分散オプティマイザー](https://pytorch.org/docs/stable/rpc.html#module-torch.distributed.optim) への入力として必要なオブジェクトであり、分散オプティマイザーは最適化する必要のあるすべてのパラメータをRRefのリストとして受け取ります。

In [ ]:
# 分散自動微分を使用し、モデルのために蓄積された勾配を取得します。
# 主に検証目的で使用します。
def get_dist_gradients(self, cid):
    grads = dist_autograd.get_gradients(cid)
    # この出力は、RPC経由でフォワードされますが、1.5.0の時点ではCPUのテンソルしか受け付けていません。
    # そのため、テンソルをGPUのメモリに出し入れしなければなりません。
    cpu_grads = {}
    for k, v in grads.items():
        k_cpu, v_cpu = k.to("cpu"), v.to("cpu")
        cpu_grads[k_cpu] = v_cpu
    return cpu_grads

# ローカルパラメーターをRRefにラップします。
# これは、リモートでパラメーターの最適化を行うDistributedOptimizerを構築する際に必要になります。
def get_param_rrefs(self):
    param_rrefs = [rpc.RRef(param) for param in self.model.parameters()]
    return param_rrefs

最後に、パラメーターサーバーを初期化するメソッドを作成します。

なお、プロセス全体にわたってパラメーターサーバーのインスタンスは一つのみであり、すべてのトレーナーが同一のパラメーターサーバーとやり取りをして、格納された同一のモデルパラメーターを更新する点に注意してください。

また、`run_parameter_server` を見てわかるように、サーバー自体は独立したアクションを行いません。

サーバーはトレーナーからのリクエストを待ち（この部分はまだ定義していません。）、リクエストされた関数を実行することで、トレーナーに応答します。

In [ ]:
# グローバルパラメーターサーバーインスタンス
param_server = None
# 一つのパラメーターサーバーであることを担保するためのロック
global_lock = Lock()


def get_parameter_server(num_gpus=0):
    """
    単一のパラメーターサーバーをすべてのトレーナープロセスに返します。
    """
    global param_server
    # ParameterServerへのハンドルを一つのみ得ることを担保します。
    with global_lock:
        if not param_server:
            # 一度だけ構築します。
            param_server = ParameterServer(num_gpus=num_gpus)
        return param_server

def run_parameter_server(rank, world_size):
    # パラメーターサーバーは、ただモデルのホストとして振る舞い、
    # トレーナーからのリクエストに応答します。
    # rpc.shutdown() は、デフォルトではすべてのワーカーが処理を完了するまで待機します。
    # 今回のケースでは、パラメーターサーバーはすべてのトレーナーが処理を完了するまで待機し、    
    # その後、処理を抜けます。
    print("PS master initializing RPC")
    rpc.init_rpc(name="parameter_server", rank=rank, world_size=world_size)
    print("RPC initialized! Running parameter server...")
    rpc.shutdown()
    print("RPC shutdown on parameter server.")

上記の実装コードでは、`rpc.shutdown()` 部分では、即時にパラメーターサーバーがシャットダウンするわけではない点に注意してください。

パラメーターサーバーは、すべてのワーカー（今回のケースではトレーナー）が `rpc.shutdown()` を呼び出すまで待機します。

これにより、すべての（まだ定義されていない）トレーナーが訓練プロセスを完了する前に、パラメータサーバーがオフラインにならないことを保証しています。


次に、`TrainerNet` クラスを定義します。

このクラスは `nn.Module` のサブクラスでもあり、`__init__` メソッドでは `rpc.remote` APIを使用してパラメーターサーバーへのRRef、あるいはリモート参照を取得します。

なお、ここではパラメーターサーバーをローカルのプロセスにコピーしていない点に注意してください。

そうではなく、 `self.param_server_rref` を、分離したプロセス上に存続するパラメーターサーバーへの分散共有ポインターとして捉えることができます。

In [ ]:
# --------- Trainers --------------------

# トレーナーによって訓練されるネットワークに対応する nn.Module
# forward() メソッドは、単に与えられたパラメーターサーバー上でネットワークを呼び出します。
class TrainerNet(nn.Module):
    def __init__(self, num_gpus=0):
        super().__init__()
        self.num_gpus = num_gpus
        self.param_server_rref = rpc.remote(
            "parameter_server", get_parameter_server, args=(num_gpus,))

次に `get_global_param_rrefs` というメソッドを定義します。

このメソッドの気持ちを理解する上では、[DistributedOptimizer](https://pytorch.org/docs/stable/rpc.html#module-torch.distributed.optim) のドキュメント、中でも特に、APIシグネチャの該当部分を読む必要があります。

オプティマイザーは、最適化の対象となるリモートのパラメーターに対応するRRefのリストを渡される必要があるため、ここで必要なRRefを取得します。



関数内の処理では、`TrainerNet` がやり取りをする唯一のリモートワーカーが `ParameterServer` であるため、単に `ParameterServer` 上で `remote_method` を呼び出しています。 

そこで、`ParameterServer` クラスで定義した `get_param_rrefs` メソッドを使用しています。このメソッドは、最適化される必要のあるパラメーターへのRRefのリストを返します。

なお、今回のケースでは、`TrainerNet` は独自のパラメーターを定義していない点に留意してください。

もし独自のパラメーターを定義した場合、各パラメーターを一つのRRefにラップし、`DistributedOptimizer` への入力に含める必要が生じます。

In [ ]:
class TrainerNet(nn.Module):
    # ...
    def get_global_param_rrefs(self):
        remote_params = remote_method(
            ParameterServer.get_param_rrefs,
            self.param_server_rref)
        return remote_params

これで（同期）RPCを呼び出し、`ParameterServer` 上で定義されたネットワークのフォワードパスを実行する `forward` メソッドを定義する準備ができました。

なお、RPCの呼び出しには `ParameterServer` へのリモートのハンドルである `self.param_server_rref` を渡している点に注意してください。

この呼び出しにより、`ParameterServer` が実行されているノード上にRPCを送信し、フォワードパスを実行し、モデルの出力に対応するテンソルを返します。

In [ ]:
class TrainerNet(nn.Module):
    # ...
    def forward(self, x):
        model_output = remote_method(
            ParameterServer.forward, self.param_server_rref, x)
        return model_output

トレーナーを完全に定義できたので、次はニューラルネットワークの訓練ループを記述し、ネットワークとオプティマイザーの作成、ネットワークを通した入力の実行、そして損失の計算をできるようにします。

訓練ループはローカルの訓練プログラムと似ている点が多くありますが、ネットワークが複数のマシン間に分散されているという性質上、少し手を加えている点があります。

下記の実装コードでは、`TrainerNet` を初期化し、`DistributedOptimizer` を構築しています。

なお、先ほど述べたように、`DistributedOptimizer` には最適化を行いたいグローバル（分散訓練に関わっているすべてのノードに渡る）パラメーターのすべてを渡す必要があります。

さらに、使用するローカルオプティマイザーを渡します。今回のケースではSGDです。

ちなみに、ローカルオプティマイザーの作成と同様に、基盤になるオプティマイザーのアルゴリズムを設定することができ、`optimizer.SGD` へのすべての引数は適切に転送されます。

例えば次のコードでは、すべてのローカルオプティマイザーで学習率として使用される独自の学習率を引数で渡しています。

In [ ]:
def run_training_loop(rank, num_gpus, train_loader, test_loader):
    # 典型的なニューラルネットワークのフォワード + バックワード + オプティマイザーステップを実行しますが、    
    # 分散に適した方法でこれらを実行します。
    net = TrainerNet(num_gpus=num_gpus)
    # DistributedOptimizerの構築
    param_rrefs = net.get_global_param_rrefs()
    opt = DistributedOptimizer(optim.SGD, param_rrefs, lr=0.03)

次に、メインとなる訓練ループを定義します。

PyTorchの [DataLoader](https://pytorch.org/docs/stable/data.html) から与えられるイテラブルオブジェクトに対してループします。

ですが、典型的なフォワード/バックワード/オプティマイザーのループを記述する前に、分散自動微分のコンテクストにロジックをラップします。



このラップは、モデルのフォワードパス内で呼び出されたRPCを記録するために必要である点に注意してください。

これにより、バックワードパス内に存在するすべての分散ワーカーを含んだ適切なグラフを構築できるようになります。

なお、分散自動微分コンテクストは、特定のイテレーションに対応する勾配を蓄積して最適化するための識別子として機能する `context_id` を返します．

ローカルワーカー上でバックワードパスを開始する典型的な `loss.backward()` の呼び出しとは違い、context_id と `loss` を引数に`dist_autograd.backward()`を呼び出しますが、この部分がバックワードパスを開始させる起点になっています。

そして、この context_id はオプティマイザーの呼び出しにも渡しますが、これは特定のバックワードパスによって計算される勾配をすべてのノードにわたって参照できるようにするために必要なid情報です。

In [ ]:
def run_training_loop(rank, num_gpus, train_loader, test_loader):
    # ...
    for i, (data, target) in enumerate(train_loader):
        with dist_autograd.context() as cid:
            model_output = net(data)
            target = target.to(model_output.device)
            loss = F.nll_loss(model_output, target)
            if i % 5 == 0:
                print(f"Rank {rank} training batch {i} loss {loss.item()}")
            dist_autograd.backward(cid, [loss])
            # 分散自動微分の実行に成功し、勾配が返されたことを確認します。
            assert remote_method(
                ParameterServer.get_dist_gradients,
                net.param_server_rref,
                cid) != {}
            opt.step(cid)

    print("Training complete!")
    print("Getting accuracy....")
    get_accuracy(test_loader, net)

次に示す実装コードは、典型的なローカルモデルと同様に、訓練完了後にモデルの正解率をただ計算しています。

しかし先ほど説明したように、この関数に与えた `net` は、`TrainerNet` のインスタンスであるため、フォワードパスは明示的にRPCを呼び出している点に留意してください。


In [ ]:
def get_accuracy(test_loader, model):
    model.eval()
    correct_sum = 0
    # GPUを使用できる場合は、GPUを使用して評価を行います。
    device = torch.device("cuda:0" if model.num_gpus > 0
        and torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            out = model(data, -1)
            pred = out.argmax(dim=1, keepdim=True)
            pred, target = pred.to(device), target.to(device)
            correct = pred.eq(target.view_as(pred)).sum().item()
            correct_sum += correct

    print(f"Accuracy {correct_sum / len(test_loader.dataset)}")

次に、RPCの初期化を担当する `ParameterServer` の主要なループとして、 `run_parameter_server` を定義した方法のように、トレーナーに関して同様のループを定義しましょう。

`run_parameter_server` との違いは、トレーナーが先ほど定義した訓練ループを実行しなければならない点です。

In [ ]:
# トレーナー用の主要なループ
def run_worker(rank, world_size, num_gpus, train_loader, test_loader):
    print(f"Worker rank {rank} initializing RPC")
    rpc.init_rpc(
        name=f"trainer_{rank}",
        rank=rank,
        world_size=world_size)

    print(f"Worker {rank} done initializing RPC")

    run_training_loop(rank, num_gpus, train_loader, test_loader)
    rpc.shutdown()

なお、`run_parameter_server` のときと同様に `rpc.shutdown()` は、ノードを抜ける前にトレーナーとパラメーターサーバーのすべてのワーカーが `rpc.shutdown()` を呼び出すまでは、デフォルトで待機する点に注意してください。


これにより、ノードが円滑に終了し、別のノードがオンラインであることを期待している間に、対象のノードがオフラインにならないようにしています。

以上でトレーナーとパラメーターサーバーの具体的な実装が完了し、残すはトレーナーとパラメーターサーバーを起動するコードを加えるのみとなりました。

初めに、パラメーターサーバーとトレーナーに適用する様々な引数を取る必要があります。

`world_size` は、訓練に関わるノード総数に対応しており、つまりすべてのトレーナーとパラメーターサーバーの合計値を表します。

また、各プロセスについての一意の `rank` も渡す必要があり、この値は `0` （単一のパラメーターサーバーを実行する場所）から `world_size -1` を取ります。

`master_addr` と `master_port` は、ランク0のプロセスがどこで実行されているかを特定する用途があり、各ノードが互いのノードを発見する際に使用されます。

この例をローカルで検証するには、単に `localhost` と生成されたすべてのインスタンスへの `master_port` を渡すことになります。

なお、説明目的のため、この例は0-2個のGPUの使用のみサポートしていますが、本チュートリアルの実装アプローチは、より多くのGPUを使用している場合であっても適用できる内容になっています。

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description="Parameter-Server RPC based training")
    parser.add_argument(
        "--world_size",
        type=int,
        default=4,
        help="""Total number of participating processes. Should be the sum of
        master node and all training nodes.""")
    parser.add_argument(
        "rank",
        type=int,
        default=None,
        help="Global rank of this process. Pass in 0 for master.")
    parser.add_argument(
        "num_gpus",
        type=int,
        default=0,
        help="""Number of GPUs to use for training, Currently supports between 0
         and 2 GPUs. Note that this argument will be passed to the parameter servers.""")
    parser.add_argument(
        "--master_addr",
        type=str,
        default="localhost",
        help="""Address of master, will default to localhost if not provided.
        Master must be able to accept network traffic on the address + port.""")
    parser.add_argument(
        "--master_port",
        type=str,
        default="29500",
        help="""Port that master is listening on, will default to 29500 if not
        provided. Master must be able to accept network traffic on the host and port.""")

    args = parser.parse_args()
    assert args.rank is not None, "must provide rank argument."
    assert args.num_gpus <= 3, f"Only 0-2 GPUs currently supported (got {args.num_gpus})."
    os.environ['MASTER_ADDR'] = args.master_addr
    os.environ["MASTER_PORT"] = args.master_port

コマンドラインの引数によって決まるパラメーターサーバー、またはトレーナーに対応するプロセスを作成するところまで完了しました。

ランクが0で渡された場合は `ParameterServer` を作成し、それ以外の場合は `TrainerNet` を作成します。

なお、実行したい関数に対応するサブプロセスを起動する際には `torch.multiprocessing` を使用し、メインスレッドから `p.join()` を呼び出して起動したサブプロセスの完了を待機する点に注意してください。

そしてトレーナーを初期化する際は、PyTorchの [データローダー](https://pytorch.org/docs/stable/data.html) を用いて、MNISTデータセット内の訓練用のデータローダーとテスト用のデータローダーを指定しています。

In [ ]:
processes = []
world_size = args.world_size
if args.rank == 0:
    p = mp.Process(target=run_parameter_server, args=(0, world_size))
    p.start()
    processes.append(p)
else:
    # 訓練に使用するデータを取得
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=32, shuffle=True,)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            '../data',
            train=False,
            transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                        ])),
        batch_size=32,
        shuffle=True,
    )
    # このノード上で訓練ワーカーを開始
    p = mp.Process(
        target=run_worker,
        args=(
            args.rank,
            world_size, args.num_gpus,
            train_loader,
            test_loader))
    p.start()
    processes.append(p)

for p in processes:
    p.join()

以上の実装例をローカルにて実行する際は、分離したターミナルのウィンドウにおいて、次のコマンドワーカーをサーバーと生成したい各ワーカーに対して実行します。

`python rpc_parameter_server.py --world_size=WORLD_SIZE --rank=RANK`

そして、例えば、ワールドサイズが2であるマスターノードに対するコマンドは、

`python rpc_parameter_server.py --world_size=2 --rank=0` 

のようになります。



また、トレーナーであれば、

`python rpc_parameter_server.py --world_size=2 --rank=1`

のコマンドを分離したウィンドウ内で行うことで、起動させることができます。

なお、本チュートリアルは、2個以下のGPUを用いて訓練することを想定していますが、この引数は `--num_gpus=N` を訓練スクリプトに与えることで設定が可能である点に留意してください。

さらに、コマンドライン引数 `--master_addr=ADDRESS` と `--master_port=PORT` を渡すことで、マスタワーカーがリスニングしているアドレスとポートを指定することもできます。

これは例えば、トレーナーとマスターノードが異なりマシン上で実行されている場合において、機能をテストする際に役立ちます。